In [1]:
import ee
import geemap
import matplotlib.pyplot as plt

In [2]:
# Authenticate to GEE
ee.Authenticate()

# Initialize the Earth Engine library
ee.Initialize(project='ee-pallavisharma')


In [3]:
roi=ee.FeatureCollection("projects/ee-pallavisharma/assets/lakhimpur_bound_part1")
# Extract the geometries from the feature collection
aoi = roi.geometry()

In [4]:
# Define the Sentinel-1 image collection
sentinel1 = ee.ImageCollection("COPERNICUS/S1_GRD")


start_date_sar='2023-07-01'
end_date_sar='2023-07-31'

# Filter by date and AOI
sentinel1_filtered = sentinel1.filterBounds(aoi).filterDate(start_date_sar,end_date_sar)
datesar = sentinel1_filtered.aggregate_array('system:time_start').getInfo()
# Check if the ROI falls under more than one tile
tiles = sentinel1_filtered.distinct('relativeOrbitNumber_start')
tile_count = tiles.size().getInfo()

if tile_count > 1:
    # Perform mosaicking if ROI falls under more than one tile
    sentinel1_mosaic = sentinel1_filtered.median()
else:
    sentinel1_mosaic = sentinel1_filtered.first()  # Select the only available tile


# Define a function to calculate RVI for each image in the collection
def calculate_rvi(image):
    vh = image.select('VV')
    vv = image.select('VH')
    return vh.subtract(vv).divide(vh.add(vv))

# Calculate RVI for each image in the Sentinel-1 collection
rvi_collection = sentinel1_filtered.map(calculate_rvi)
# Calculate RVI for the mosaic image
rvi_mosaic = calculate_rvi(sentinel1_mosaic).clip(roi)


In [5]:

# Define the Sentinel-2 image collection
sentinel2 = ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED")

# Filter by date and AOI
sentinel2_filtered = sentinel2.filterBounds(aoi).filterDate('2023-07-01', '2023-07-31')
# Create a list of dates from the image collection
dates = sentinel2_filtered.aggregate_array('system:time_start').getInfo()

# Perform mosaicking by merging all tiles that intersect the ROI
sentinel2_mosaic = sentinel2_filtered.mosaic()

# Define a function to calculate NDVI for each image in the collection
def calculate_ndvi(image):
    red = image.select('B4')
    nir = image.select('B8')
    return nir.subtract(red).divide(nir.add(red))

# Calculate NDVI for the mosaic image
ndvi_mosaic = calculate_ndvi(sentinel2_mosaic).clip(roi)
# Calculate NDVI for each image in the Sentinel-2 collection
ndvi_collection = sentinel2_filtered.map(calculate_ndvi)

In [6]:
# Iterate over the dates and add NDVI and RVI as separate layers
Map = geemap.Map()
Map.centerObject(roi, 18)  # Center the map on your ROI


# Create an empty list to store NDVI images and dates
ndvi_images = []
dates = []

# Iterate over the Sentinel-2 tiles
tile_list = sentinel2_filtered.toList(sentinel2_filtered.size())
for i in range(tile_list.size().getInfo()):
    tile = ee.Image(tile_list.get(i))

    # Get the date for the image
    date = ee.Date(tile.get('system:time_start')).format('YYYY-MM-dd').getInfo()
    dates.append(date)

    # Calculate NDVI for the tile and clip it to the ROI
    ndvi_tile = calculate_ndvi(tile).clip(roi)

    # Append the NDVI tile to the list
    ndvi_images.append(ndvi_tile)

# Add NDVI layers for each date to the map
for i, date in enumerate(dates):
    Map.addLayer(
        ndvi_images[i],
        {'min': -1, 'max': 1, 'palette': ['brown', 'red', 'green']},
        f'NDVI ({date})'
    )

    #Map.addLayer(rvi_image, {'min': -1, 'max': 1, 'palette': ['red', 'white', 'green']}, 'Radar Vegetation Index (Sentinel-1)'+str(date))


# Create an empty list to store RVI images and dates
rvi_images = []
datesar = []

# Iterate over the Sentinel-1 tiles
tile_list = sentinel1_filtered.toList(sentinel1_filtered.size())
for i in range(tile_list.size().getInfo()):
    tile = ee.Image(tile_list.get(i))

    # Get the date for the image
    date = ee.Date(tile.get('system:time_start')).format('YYYY-MM-dd').getInfo()
    datesar.append(date)

    # Calculate RVI for the tile and clip it to the ROI
    rvi_tile = calculate_rvi(tile).clip(aoi)

    # Append the RVI tile to the list
    rvi_images.append(rvi_tile)

for i, date in enumerate(datesar):
    Map.addLayer(
        rvi_images[i],
        {'min': -1, 'max': 1, 'palette': ['brown', 'red', 'green']},
        f'RVI ({date})'
    )

# Create a manual legend
legend_html = """
<div style="position: absolute; bottom: 8px; left: 8px; padding: 5px; background: rgba(255, 255, 255, 0.7); border: 1px solid #ccc; z-index: 100; font-size: 12px;">
    <div><b>Legend</b></div>
    <div><span style="color: blue;">Radar Vegetation Index (Sentinel-1)</span></div>
    <div><span style="color: brown;">NDVI (Sentinel-2)</span></div>
</div>
"""
Map.add_html(legend_html)

# Display the map
Map





Map(center=[27.901939258112264, 81.09986033576905], controls=(WidgetControl(options=['position', 'transparent_…